In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from matplotlib import pyplot as plt

In [2]:
species="mouse"
gsm4sig_version = 1

In [3]:
bulk_study_meta = pd.read_csv(species+"_bulk_study_meta.csv", index_col=0)
frequent_terms = pd.read_csv("human_frequent_terms.csv")

In [6]:
frequent_terms

,Label,Frequency,Category
0,rep,23641,0
1,rna,16840,0
2,seq,13681,0
3,replicate,6047,0
4,control,6023,1
...,...,...,...
500,cutll,132,0
501,tp,132,0
502,wildtype,132,1
503,peripheral,132,0


In [5]:
frequent_terms.at[504, "Label"] = "null"

In [7]:
bulk_study_meta["sample_title_set"] = (bulk_study_meta["sample_title"]
                                       .str.lower().str.findall(r"[a-z]+")
                                       .apply(frozenset))

ctrl_set = set(frequent_terms.loc[frequent_terms["Category"] == 1]["Label"])
pert_set = set(frequent_terms.loc[frequent_terms["Category"] == 2]["Label"])

In [8]:
bulk_study_meta

,series_id,geo_accession,singlecellprobability,sample_title,h5_idx,sample_title_set
0,GSE54390,GSM1314260,0.185995,TRACK 43-75 kidney 1,0,"(kidney, track)"
1,GSE64027,GSM1563082,0.186791,20NNK_LPS_Sil_I3C9,3,"(c, sil, i, nnk, lps)"
2,GSE36025\tGSE49417\tGSE49847,GSM1000563,0.163561,CSHL_RnaSeq_Cortex_adult-8wks (superseded by G...,4,"(gse, wks, rnaseq, cortex, cshl, superseded, a..."
3,GSE44157\tGSE44163,GSM1079794,0.143830,KO_siNr6a1_R0,13,"(r, sinr, a, ko)"
4,GSE49906,GSM1223636,0.133423,RNA-seq ventricle PN28,14,"(pn, seq, ventricle, rna)"
...,...,...,...,...,...,...
130007,GSE165318,GSM5031527,0.212547,S15,405635,(s)
130008,GSE165318,GSM5031528,0.182000,S17,405636,(s)
130009,GSE165318,GSM5031529,0.239316,S37,405637,(s)
130010,GSE165318,GSM5031530,0.256362,S38,405638,(s)


In [9]:
def categorize_cp(test_set, ctrl_set = ctrl_set, pert_set = pert_set): # categorizes sample as ctrl or pert
    difference = len(test_set & ctrl_set) - len(test_set & pert_set)   # 0 - undetermined
    if difference == 0: return 0                                       # 1 - control group
    return 1 if difference > 0 else 2                                  # 2 - perturbation group

In [10]:
bulk_study_meta["cp_group"] = bulk_study_meta["sample_title_set"].apply(categorize_cp)

In [11]:
len(bulk_study_meta["series_id"].unique())

8971

In [12]:
no_undet = []
for gse in bulk_study_meta.groupby("series_id")["sample_title_set"]:                  
    if (#0 not in np.array(gse[1])  and                                               
        #len(gse[1].value_counts()) <= 2 and              
        (gse[1].value_counts() >= 2).all()): no_undet.append(gse[0])
        
no_undet_study = bulk_study_meta[bulk_study_meta["series_id"].isin(no_undet)].copy()

In [13]:
len(no_undet)

6832

In [ ]:
#gse_per_term = []                                                                              # counts unique gse for each 
#def det_gse(test_set, term): return 1 if len(test_set & {term}) > 0 else 0                     # control or perturbation term
#for i in frequent_terms["Label"]:
#    bulk_study_meta["det_gse"] = bulk_study_meta["sample_title_set"].apply(det_gse, term=i)
#    gse_per_term.append(len(bulk_study_meta.loc[bulk_study_meta["det_gse"] == 1]["series_id"].unique()))
#    bulk_study_meta["det_gse"] = np.zeros(162372)
    
#frequent_terms["num_gse"] = gse_per_term
#frequent_terms.to_csv(species+"_frequent_terms_gse_count.csv", index=False)

In [ ]:
#def gsm_title_cluster(series, cluster_esp = 0.01, cluster_min_samp = 2):
#    mlb = MultiLabelBinarizer()
#    term_mat = mlb.fit_transform(series["sample_title_set"])
#    if 1 in term_mat.shape: return np.zeros(len(series)).astype(int)
#    pca = PCA()
#    pca.fit(term_mat.T)
#    clustering = DBSCAN(eps=0.01, min_samples=2)
#    #print(list(zip(pca.components_[0], pca.components_[1])))
#    #clustering.fit(list(zip(pca.components_[0], pca.components_[1])))
#    clustering.fit(pca.components_[0].reshape(-1, 1))
#    return clustering.labels_

In [ ]:
# for gse in no_undet_study["series_id"].unique():
#    series = no_undet_study.loc[no_undet_study["series_id"] == gse]
#    no_undet_study.loc[no_undet_study["series_id"] == gse, "gsm_cluster"] = gsm_title_cluster(series)

In [ ]:
#no_undet_study["gsm_cluster"] = no_undet_study["sample_title_set"].apply(hash)
#sum(no_undet_study.groupby("gsm_cluster")["sample_title_set"].unique().apply(len) - 1) # return 0 if no hash collisions

In [14]:
def count_series(series):
    series_counter = (Counter(list(zip(*series.groupby(["cp_group", "sample_title_set"])
                                       .groups.keys()))[0]))
    return (min(series_counter[0], 3), 
            min(series_counter[1], 3), 
            min(series_counter[2], 3)) # subgroup counts for undet, ctrl, pert group, respectively
    

# def categorize_gse_type(series):
#     #undet_ct, ctrl_ct, pert_ct = count_series(series)
#     gse_type = {(i, j, k):n for i in range(4) for j in range(4) for k in range(4) for n in range(64)}
#     print(gse_type)
    
# def categorize_gse_type(series):
#     undet_ct, ctrl_ct, pert_ct = count_series(series)
#     if ctrl_ct == 0: 
#         if pert_ct == 2: return 2               # type  2:  2 groups, no ctrl(s)
#         if pert_ct >= 3: return 5               # type  5: >2 groups, no ctrl(s)
#     if ctrl_ct == 1:
#         if pert_ct == 1: return 1               # type  1:  2 groups,  1 ctrl(s)
#         if pert_ct >= 2: return 3               # type  3: >2 groups,  1 ctrl(s)
#     if ctrl_ct >= 2:
#         if pert_ct >= 1: return 4               # type  4: >2 groups, >2 ctrl(s)
#     return -1                                   # type -1:  erratic

In [15]:
gse_types = {}
for n, gse in enumerate(no_undet_study["series_id"].unique()):
    if n % 1000 == 0: print(n, "done")
    series = no_undet_study.loc[no_undet_study["series_id"] == gse]
    gse_types[gse] = count_series(series)
    
categorized_gse = pd.DataFrame({"gse":gse_types.keys(), "gse_type":gse_types.values()})

0 done
1000 done
2000 done
3000 done
4000 done
5000 done
6000 done


In [ ]:
categorized_gse.loc[categorized_gse["gse_type"] == (1, 0, 1)]

In [ ]:
no_undet_study.loc[no_undet_study.series_id == "GSE133674"].sort_values(["cp_group", "sample_title_set"])

In [16]:
print("(?, c, p)    freq   3 := >2")
print(categorized_gse["gse_type"].value_counts().to_string())

(?, c, p)    freq   3 := >2
(3, 0, 0)    1053
(2, 0, 0)     959
(1, 1, 0)     883
(0, 1, 1)     768
(1, 0, 0)     606
(2, 2, 0)     291
(2, 1, 0)     228
(0, 2, 2)     192
(3, 1, 0)     185
(3, 3, 0)     166
(1, 0, 1)     149
(3, 2, 0)     121
(0, 3, 3)      92
(1, 1, 1)      80
(0, 2, 0)      68
(2, 1, 1)      66
(1, 2, 1)      65
(1, 3, 0)      56
(0, 0, 2)      52
(0, 1, 2)      51
(2, 0, 2)      48
(0, 0, 3)      41
(1, 1, 2)      41
(1, 2, 0)      39
(0, 1, 0)      36
(3, 0, 1)      36
(0, 3, 0)      36
(2, 3, 0)      29
(2, 0, 1)      26
(3, 1, 1)      23
(3, 0, 3)      23
(0, 0, 1)      21
(3, 3, 3)      20
(0, 2, 1)      17
(2, 2, 2)      16
(1, 0, 2)      14
(3, 0, 2)      14
(3, 3, 1)      14
(0, 2, 3)      14
(0, 1, 3)      13
(1, 3, 2)      13
(1, 0, 3)      12
(1, 3, 3)      12
(1, 1, 3)      10
(0, 3, 1)      10
(2, 0, 3)      10
(3, 2, 2)       9
(1, 2, 3)       9
(3, 2, 1)       9
(3, 3, 2)       8
(3, 2, 3)       8
(2, 3, 2)       8
(1, 2, 2)       8
(2, 3, 1)       8


In [ ]:
# no_undet_study["gse_type"] = np.zeros(len(no_undet_study)).astype(int)

# for gse in no_undet_study["series_id"].unique():
#     series = no_undet_study.loc[no_undet_study["series_id"] == gse]
#     no_undet_study.loc[no_undet_study["series_id"] == gse, "gse_type"] = np.repeat(categorize_gse_type(series), len(series))

In [17]:
def extrap_score(study_cat):
    #penalize inferred type (ctrl, pert), direction, --pairing--
    undet, ctrl, pert = study_cat
    score = 0
    if undet == 0 or pert == 0:                 #(0, x, y) or (y, x, 0)
        if ctrl == 1:                           #(0, 1, y) or (y, 1, 0)
            if undet != 0: score += 1           #penalty for inferred pert
        if ctrl < 1:                            #(0, 0, y) or (y, 0, 0)
            score += 1                          #penalty for inferred ctrl
            score += 2 if undet != 0 else 1     #penalty for inferred direction and/or inferred pert
    else:
        if ctrl < 1:                            #(x, 0, y)
            score += 1 if undet == 1 else -1    #penality for inferred ctrl
        else:
            score = -1                          #unaccounted for
    return score
        

def to_calc_sig_format(study_cat, studies = no_undet_study, categorized_gse = categorized_gse):
    undet, ctrl, pert = study_cat
    study_group = studies.loc[studies.series_id.isin(
        categorized_gse.loc[categorized_gse["gse_type"] == study_cat]["gse"])].copy()
    if ctrl == 0 and undet == 1: #(1, 0, x)
        study_group.replace({'cp_group':{0:"ctrl_gsm", 2:"pert_gsm"}}, inplace=True)
    else:
        study_group.replace({'cp_group':{1:"ctrl_gsm", 0:"pert_gsm", 2:"pert_gsm"}}, inplace=True)
        
    if ctrl == 0 and (undet == 0 or pert == 0):
        study_cp_series = study_group.groupby(["series_id", "sample_title_set"])["cp_group"].unique()
        new_ctrl_idx = study_cp_series.index[::2]
        study_cp_series[study_cp_series.index.isin(new_ctrl_idx)] = "ctrl_gsm"
        study_group_map = dict(study_cp_series.explode())
        study_group = (study_group.assign(cp_group=study_group[["series_id","sample_title_set"]]
                                         .apply(tuple, axis=1).map(study_group_map)))
            
        
    return (study_group
            .groupby(["series_id", "cp_group","sample_title_set"])["geo_accession"]
            .aggregate(lambda x: list(x))
            #.aggregate(lambda x: "|".join(x))
            .unstack()
            .apply(lambda x: list(x.dropna()), axis=1)
            .unstack()
            .explode("ctrl_gsm")
            .explode("pert_gsm")
            .assign(extrap_score=extrap_score(study_cat)))

In [18]:
valid_types = [(0, 1, 1), (0, 1, 2), (0, 1, 3), 
               (1, 1, 0), (2, 1, 0), (3, 1, 0), 
               (0, 0, 2), (2, 0, 0),
               (1, 0, 1), (1, 0, 2), (1, 0, 3)]
gsm4sig = pd.concat([to_calc_sig_format(i) for i in valid_types]).reset_index()
gsm4sig["series_id"] = gsm4sig["series_id"].replace(regex=r"\t", value="-")
gsm4sig.to_csv(f"{species}_gsm4sig_v{gsm4sig_version}.csv")

In [ ]:
len(gsm4sig)